<a href="https://colab.research.google.com/github/PrathibhaNishadi/Cricket-Squad-Selector/blob/main/T20Bat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.optimizers import Adam
import keras
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import locale
import os
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import numpy as np
import glob
import cv2
import os
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model


def load_data_attributes(inputPath):
	df = pd.read_csv(inputPath)
	return df

def process_data_attributes(df, train, test):
  continuous = ["TMatches","Avg", "SR", "Tboun"]
  trainX = (train[continuous])
  testX = (test[continuous])
  return (trainX, testX)

def create_mlp(dim, regress=False):
	model = Sequential()
	model.add(Dense(13, input_dim=4, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))

	if regress:
		model.add(Dense(1, activation="linear"))

	return model


print("[INFO] loading Cricket attributes...")

df = load_data_attributes("drive/My Drive/Project/TTBatTrain.csv")

print("[INFO] constructing training/testing split...")
(train, test) = train_test_split(df, test_size=0.10, random_state=42)
_trainY = train["Runs"]
_testY = test["Runs"]
trainY=[]

for i in _trainY:
  trainY.append(i)
print(trainY)
testY=[]

for i in _testY:
  testY.append(i)
print(testY)

trainY=np.array(trainY)
testY=np.array(testY)

print("[INFO] processing data...")
(trainX, testX) = process_data_attributes(df, train, test)

model = create_mlp(trainX.shape[1], regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss='mean_squared_error', optimizer='adam')


print("[INFO] training model...")

model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=200, batch_size=8)

model.save('TTBat_model.mod')


In [ ]:
import csv
import keras
import pandas as pd

predict=pd.read_csv("drive/My Drive/Project/TTBatting.csv")
labels = ["Matches","BatAvg", "StrikeRate", "Boundaries"]
predictList=(predict[labels])


model=keras.models.load_model('TTBat_model.mod')
results=model.predict(predictList)
results=results

output = results.tolist()
names = (predict["Name"])
types = (predict["Type"])

myFile = open('TTBatResults.csv', 'w')   
with myFile:  
    myFields = ['Name','Type' ,'Runs']
    writer = csv.DictWriter(myFile, fieldnames=myFields)    
    writer.writeheader()
    
    for i in range(len(output)):
      writer.writerow({'Name' : names[i], 'Type' : types[i], 'Runs': output[i][0]})
#       print (names[i] , end=" ")
#       print (output[i][0])
      
# making data frame from csv file 
data = pd.read_csv("TTBatResults.csv")

# creating a rank column and passing the returned rank series 
data["Rank"] = data["Runs"].rank(ascending=0)

# sorting w.r.t name column 
data.sort_values("Rank", inplace = True)  

export_csv = data.to_csv ('TTBatResults.csv', index = None, header=True)

data


,Name,Type,Runs,Rank
0,Thisara Perera,Allrounder,22.545994,1.0
12,Isuru Udana,Allrounder,21.813961,2.0
2,Niroshan Dickwella,Wicketkeeper batsman,21.712172,3.0
5,Kamindu Mendis,Allrounder,18.385475,4.0
4,Dhananjaya de Silva,Allrounder,16.343407,5.0
11,Angelo Perera,Allrounder,14.369973,6.0
1,Dinesh Chandimal,Wicketkeeper batsman,13.993379,7.0
15,Dasun Shanaka,Allrounder,13.422421,8.0
3,Kusal Perera,Wicketkeeper batsman,11.397746,9.0
17,Suranga Lakmal,Bowler,10.592761,11.5
